# Documents DB File

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"
file_name = "../data/raw/fintech-scopus.csv"
import_scopus_file(file_name, directory)

# !cat ../data/processed/debug/filter.yaml

# Thesaurus

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# clean_institutions(directory)
# clean_keywords(directory)
# apply_thesaurus(directory, thesaurus_file="keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus", strict=False)
# create_thesaurus(directory, column="keywords", thesaurus_file="test_thesaurus.txt", sep="; ")

# Basic analysis

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# core_authors(directory)
# core_sources(directory)
# count_documents_by_term(directory, column='authors', min_occ=6)
# coverage(directory)
# impact_analysis(directory, column="authors")
# summary(directory)
# terms_analysis(directory, column='authors', min_occ=5)
# terms_per_year_analysis(directory, column='authors', metric='num_documents', min_occ=5)
# time_analysis(directory)
# top_documents(directory)

# Basic plots

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# bar_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# pie_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# column_chart(series=time_analysis(directory)['num_documents'], darkness=time_analysis(directory)['global_citations'])
# word_cloud(terms_analysis(directory, column='authors')['num_documents'].head(50), cmap='Reds')
# tree_map(terms_analysis(directory, column='authors')['num_documents'].head(50), cmap='Reds')
# heat_map(terms_per_year_analysis(directory, column='authors', metric='num_documents', min_occ=5), cmap='Reds')


# Basic analytics charts

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"

# bradford_plot(directory)
# lotka_plot(directory)
# worldmap(directory, metric="num_documents")

# Advanced analysis

In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

directory = "../data/processed/debug"


# auto_corr_matrix(directory, column='authors', min_occ=5)
# co_occurrence_matrix(directory, column='authors', min_occ=5)
bubble_chart(co_occurrence_matrix(directory, column='authors', min_occ=5))
# cross_corr_matrix(directory, column='authors', by='author_keywords', min_occ=5)

# BigraphAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', by='countries').table_

,,authors,Rabbani MR,Arner DW,Wojcik D,Buckley RP,Reyes-Mercado P,Wonglimpiyarat J,Gozman DP,Serrano W,Khan S,Schwienbacher A,Ozili PK,Tan B,Zetzsche DA,Hamdan A,Nieves EH,Baber H,Mention A-L,Faccia A,Ashta A
,,#nd,10,9,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,5,5
,,#tc,69,135,49,132,0,52,26,15,49,50,151,105,44,18,15,12,35,27,9
authors,#nd,#tc,,,,,,,,,,,,,,,,,,,
Rabbani MR,10,69,1.000000,0.308533,0.425376,0.301425,0.0,0.332847,0.449809,0.280687,0.852789,0.421522,0.303654,0.361884,0.309238,0.488507,0.433319,0.482861,0.141109,0.377563,0.432752
Arner DW,9,135,0.308533,1.000000,0.491232,0.966728,0.0,0.307780,0.512535,0.229893,0.283288,0.428775,0.397770,0.368122,0.937234,0.376547,0.337601,0.363355,0.148925,0.337708,0.350182
Wojcik D,7,49,0.425376,0.491232,1.000000,0.452474,0.0,0.498510,0.596941,0.327390,0.377788,0.569704,0.507067,0.463196,0.463851,0.550864,0.461128,0.500571,0.195483,0.447670,0.487258
Buckley RP,7,132,0.301425,0.966728,0.452474,1.000000,0.0,0.270629,0.481020,0.222771,0.278894,0.411943,0.368707,0.353665,0.976140,0.364924,0.330385,0.361723,0.154972,0.319976,0.345759
Reyes-Mercado P,7,0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Wonglimpiyarat J,6,52,0.332847,0.307780,0.498510,0.270629,0.0,1.000000,0.443474,0.258308,0.307671,0.473546,0.360975,0.338194,0.277576,0.442879,0.346421,0.381024,0.112868,0.353172,0.398547
Gozman DP,6,26,0.449809,0.512535,0.596941,0.481020,0.0,0.443474,1.000000,0.336650,0.388482,0.539025,0.442244,0.603034,0.493061,0.550444,0.474061,0.489554,0.182040,0.447393,0.489554


In [ ]:
import pandas as pd

x = pd.Series(['a/0', 'b/1', 'c/2', 'd/3'])
x.str.replace('/\d+', "", regex=True)

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/documents.csv").iso_source_name.value_counts()

In [ ]:
pd.read_csv("../data/processed/debug/documents.csv").columns